In [1]:
from scripts import setup_environment

setup_environment()

In [2]:
import textwrap

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)


def get_system_prompt(template_name: str):
    # System message templates (priming)
    system_templates = {
        "step_by_step": "You are a meticulous problem-solver. Analyze silently and return ONLY the correct answer text.",
        "creative": "You excel at lateral thinking. Treat this as a riddle. Return ONLY the option text.",
        "elimination": "Eliminate wrong options internally. Return ONLY the text of the correct answer.",
        "metaphor": "Interpret keywords metaphorically. Return ONLY the correct answer text.",
        "confidence": "Score options internally. Return ONLY the highest-scoring answer text.",
        "perspective_shift": "Analyze through multiple perspectives silently. Return ONLY the answer text.",
        "common_sense": "Combine logic and creativity. Return ONLY the correct answer text.",
        "assumption_challenge": "Challenge hidden assumptions internally. Return ONLY the answer text.",
        "pattern_matching": "Find patterns silently. Return ONLY the correct answer text.",
        "intuitive": "Critique your intuition internally. Return ONLY the final answer text.",
    }

    system_prompt = system_templates[template_name]
    system_prompt = textwrap.dedent(system_prompt)

    system_prompt_template = SystemMessagePromptTemplate.from_template(
        system_prompt, id=template_name
    )
    return system_prompt_template


def get_user_prompt():
    prompt = """
    Question: {question}
    Choices:
    {choices}
    """

    prompt = textwrap.dedent(prompt)

    prompt_template = HumanMessagePromptTemplate.from_template(prompt)
    return prompt_template


def create_prompt_template(
    system_prompt_template_name: str = "step_by_step",
):
    system_prompt_template = get_system_prompt(system_prompt_template_name)
    user_prompt_template = get_user_prompt()
    chat_prompt_template = ChatPromptTemplate.from_messages(
        [system_prompt_template, user_prompt_template]
    )

    return chat_prompt_template

In [3]:
from scripts.dataset import BrainteaserDataset

dataset = BrainteaserDataset("data")

In [4]:
from scripts.dataset import RiddleQuestion

chat_prompt_template = create_prompt_template("step_by_step")


def args_generator(riddle_question: RiddleQuestion):
    template_args = {
        "question": riddle_question.question,
        "choices": "\n".join(
            [
                f"Option {j + 1}: {choice}"
                for j, choice in enumerate(riddle_question.choice_list)
            ]
        ),
    }

    return template_args

In [5]:
from scripts.executor import Executor

executor = Executor(
    models=[
        "meta-llama/Llama-3.2-1B-Instruct",
    ],
    riddle_dataset=dataset.sp_train[:3],
)

results = executor.execute(chat_prompt_template, args_generator)

2025-01-28 23:28:17,386 - INFO - Initialized executor with 1 models and 3 riddles
2025-01-28 23:28:17,387 - INFO - Starting execution
2025-01-28 23:28:17,387 - INFO - Processing meta-llama/Llama-3.2-1B-Instruct
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-01-28 23:28:20,377 - INFO - meta-llama/Llama-3.2-1B-Instruct took 1s
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-01-28 23:28:24,037 - INFO - meta-llama/Llama-3.2-1B-Instruct took 3s
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-01-28 23:28:25,831 - INFO - meta-llama/Llama-3.2-1B-Instruct took 1s
2025-01-28 23:28:25,831 - INFO - Completed meta-llama/Llama-3.2-1B-Instruct
2025-01-28 23:28:25,846 - INFO - Persisting results...
2025-01-28 23:28:25,848 - INFO - Results saved to results\all_results.pkl
2025-01-28 23:28:25,848 - INFO - Execution completed


In [9]:
for model, model_results in results.items():
    print(f"Model: {model}")
    for result in model_results:
        print(f"Prompt: {result.prompt_used}")
        print(f"Output: {result.model_output}")
        print("-" * 50)

Model: meta-llama/Llama-3.2-1B-Instruct
Prompt: {'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?', 'choices': 'Option 1: Some daughters get married and have their own family.\nOption 2: Each daughter shares the same brother.\nOption 3: Some brothers were not loved by family and moved away.\nOption 4: None of above.'}
Output: System: You are a meticulous problem-solver. Analyze silently and return ONLY the correct answer text.
Human: 
Question: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?
Choices:
Option 1: Some daughters get married and have their own family.
Option 2: Each daughter shares the same brother.
Option 3: Some brothers were not loved by family and moved away.
Option 4: None of above.

AI: None of the above.
--------------------------------------------------
Prompt: {'question':